# Polars vs Pandas: introducción rápida


## 1. Instalación rápida
```bash
pip install polars pandas
```
Polars distribuye binarios optimizados (Rust); la instalación suele ser < 10 s.

In [3]:
import pandas as pd
import polars as pl
import numpy as np

# Datos de ejemplo
np.random.seed(42)
pdf = pd.DataFrame({
    'grupo': np.random.choice(list('ABC'), 10),
    'valor': np.random.randint(1, 20, 10),
    'fecha': pd.date_range('2025-01-01', periods=10, freq='D')
})
print('Pandas')
display(pdf)

pldf = pl.from_pandas(pdf)
print('Polars')
pldf

Pandas


,grupo,valor,fecha
0,C,11,2025-01-01
1,A,11,2025-01-02
2,C,4,2025-01-03
3,C,8,2025-01-04
4,A,3,2025-01-05
5,A,2,2025-01-06
6,C,12,2025-01-07
7,B,6,2025-01-08
8,C,2,2025-01-09
9,C,1,2025-01-10


Polars


grupo,valor,fecha
str,i64,datetime[ns]
"""C""",11,2025-01-01 00:00:00
"""A""",11,2025-01-02 00:00:00
"""C""",4,2025-01-03 00:00:00
"""C""",8,2025-01-04 00:00:00
"""A""",3,2025-01-05 00:00:00
"""A""",2,2025-01-06 00:00:00
"""C""",12,2025-01-07 00:00:00
"""B""",6,2025-01-08 00:00:00
"""C""",2,2025-01-09 00:00:00


## 2. Filtrado simple
### Pandas
```python
pdf[pdf['valor'] > 10]
```
### Polars (eager)
```python
pldf.filter(pl.col('valor') > 10)
```

In [5]:
%time
pandas_filtrado = pdf[pdf['valor'] > 10]

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 7.87 µs


In [6]:
%time
polars_filtrado = pldf.filter(pl.col('valor') > 10)


CPU times: user 1 µs, sys: 1 µs, total: 2 µs
Wall time: 4.05 µs


In [7]:
pandas_filtrado, polars_filtrado

(  grupo  valor      fecha
 0     C     11 2025-01-01
 1     A     11 2025-01-02
 6     C     12 2025-01-07,
 shape: (3, 3)
 ┌───────┬───────┬─────────────────────┐
 │ grupo ┆ valor ┆ fecha               │
 │ ---   ┆ ---   ┆ ---                 │
 │ str   ┆ i64   ┆ datetime[ns]        │
 ╞═══════╪═══════╪═════════════════════╡
 │ C     ┆ 11    ┆ 2025-01-01 00:00:00 │
 │ A     ┆ 11    ┆ 2025-01-02 00:00:00 │
 │ C     ┆ 12    ┆ 2025-01-07 00:00:00 │
 └───────┴───────┴─────────────────────┘)

## 3. Agregación `groupby`
### Pandas
```python
pdf.groupby('grupo')['valor'].mean()
```
### Polars
```python
pldf.groupby('grupo').agg(pl.col('valor').mean())
```

In [8]:
%time
pandas_grouped = pdf.groupby('grupo')['valor'].mean()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


In [10]:
%time
# Ensure pldf is a Polars DataFrame
if not isinstance(pldf, pl.DataFrame):
	pldf = pl.from_pandas(pdf)
polars_grouped = pldf.groupby('grupo').agg(pl.col('valor').mean())

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.11 µs


AttributeError: 'DataFrame' object has no attribute 'groupby'

In [ ]:
pandas_grouped, polars_grouped

## 4. Lazy pipeline (ventaja de Polars)
```python
resultado = (
    pldf.lazy()
        .with_columns((pl.col('valor') * 2).alias('valor_x2'))
        .filter(pl.col('valor_x2') < 25)
        .collect()
)
```
En Pandas cada paso se evalúa inmediatamente; Polars optimiza toda la cadena antes de ejecutarla.

In [11]:
resultado = (
    pldf.lazy()
        .with_columns((pl.col('valor') * 2).alias('valor_x2'))
        .filter(pl.col('valor_x2') < 25)
        .collect()
)
resultado

grupo,valor,fecha,valor_x2
str,i64,datetime[ns],i64
"""C""",11,2025-01-01 00:00:00,22
"""A""",11,2025-01-02 00:00:00,22
"""C""",4,2025-01-03 00:00:00,8
"""C""",8,2025-01-04 00:00:00,16
"""A""",3,2025-01-05 00:00:00,6
"""A""",2,2025-01-06 00:00:00,4
"""C""",12,2025-01-07 00:00:00,24
"""B""",6,2025-01-08 00:00:00,12
"""C""",2,2025-01-09 00:00:00,4


## 5. Diferencias clave
- **Motor**: Polars (Rust, multihilo) vs Pandas (C/NumPy, un solo hilo).
- **Lazy evaluation**: Optativo en Polars.
- **Sintaxis basada en expresiones**: `pl.col`, más parecida a SQL o Spark.
- **Memoria y velocidad**: Polars suele ser 2‑10× más rápido y eficiente.
- **Manejo de valores nulos**: Polars distingue `null` y `NaN`.
- **Ecosistema**: Pandas tiene más extensiones; Polars evoluciona rápido.

## 6. Recursos adicionales
- [Documentación oficial de Polars](https://pola.rs/)
- [Guía de migración Pandas → Polars](https://pola.rs/guide/migration/)
- Benchmarks: <https://github.com/pola-rs/polars-benchmarks>